In [1]:
import requests
import pandas as pd
import numpy as np
import dill


In [2]:
#load home attributes data

with open(r'df_filtered.pkd', 'rb') as f:
   #home_list.extend(dill.load(f))
   df_home = dill.load(f)  

In [ ]:
with open(r'df_demo.pkd', 'rb') as f:
   #home_list.extend(dill.load(f))
   df_home = dill.load(f)  

In [ ]:
def getFips(df):
    
    url = r'https://geo.fcc.gov/api/census/block/find'
    fips_list = []
    latlong_df = df[['latitude', 'longitude']]
    
    for i in range(len(df)):
        lat = latlong_df.iloc[i, 0]
        lng = latlong_df.iloc[i, 1]
        response = requests.get(url, params={'latitude': lat, 
                                         'longitude': lng,
                                         'censusYear': 2020,
                                         'format': 'json'
                                        })
    
        fips = response.json()['Block']['FIPS']
        list_item = (df.index[i], fips)
        fips_list.append(list_item)
    return fips_list
    

In [ ]:
fips_list = getFips(df)

# construct fips df
fips_df = pd.DataFrame(fips_list, columns=['Index', 'fips'])
fips_df=fips_df.set_index('Index')
fips_df['fipsTrim'] = fips_df['fips'].str[:-4]
fips_df

In [ ]:
# merge fips data onto home attributes df

df_home_fips = df_home.copy(deep=True)
df_home_fips = df_fips.merge(fips_df, how='inner', left_index=True, right_index=True)


In [ ]:
#Merge demo data onto home attributes data

df_combined=df_home_fips.merge(df_demo, on='fipsTrim', how='left')